In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pandas as pd
import numpy as np 
import tensorflow as tf
import seaborn as sns
import pickle as pkl
import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder
import sklearn 

from datetime import datetime

In [3]:
# Display

pd.set_option('max_colwidth', 800)
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
df = pd.read_csv('./dataset/train/train.csv')

In [5]:
df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b752eebc9b50313e152,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9acf56eb3b04793d9ae5,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce,7.706428
1,03930f033646d073462b35d411616323597715ac4fc3982021ff25387177a782,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9acf56eb3b04793d9ae5,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac8f9e9a2596173f80c9,6.662563
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf11ea2bb87c4b0100a4,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9acf56eb3b04793d9ae5,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da5045d77f3b4185d8f790,7.871602
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04782dcd41548b3974b5,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9acf56eb3b04793d9ae5,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35,5.344943
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb294ba12c85962b0a9842,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9acf56eb3b04793d9ae5,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35,7.059346


In [6]:
df = df.drop(['reservation_id','memberid'],axis=1)

In [7]:
# selecting object column 

object_col_li = df.select_dtypes(include='object').columns

object_col_li = object_col_li[3:]
object_col_li

Index(['member_age_buckets', 'cluster_code', 'reservationstatusid_code',
       'resort_id'],
      dtype='object')

In [8]:
from collections import defaultdict

le = defaultdict(LabelEncoder)

for col in object_col_li:
    df[col] = le[col].fit_transform(df[col])

df.head()

,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,5,1,5,2,7,7.706428
1,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,5,1,5,0,1,6.662563
2,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,5,1,4,0,9,7.871602
3,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,5,1,3,0,25,5.344943
4,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,5,1,3,0,25,7.059346


In [9]:
le

defaultdict(sklearn.preprocessing.label.LabelEncoder,
            {'member_age_buckets': LabelEncoder(),
             'cluster_code': LabelEncoder(),
             'reservationstatusid_code': LabelEncoder(),
             'resort_id': LabelEncoder()})

In [10]:
for key,val in le.items():
    print(val.classes_)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']
['A' 'B' 'C' 'D' 'E' 'F']
['A' 'B' 'C' 'D']
['0b918943df0962bc7a1824c0555a389347b4febdc7cf9d1254406d80ce44e3f9'
 '39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac8f9e9a2596173f80c9'
 '3e1e967e9b793e908f8eae83c74dba9bcccce6a5535b4b462bd9994537bfe15c'
 '3fdba35f04dc8c462986c992bcf875546257113072a909c162f7e470e581e278'
 '48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c111b49baf362511afc'
 '49d180ecf56132819571bf39d9b7b342522a2ac6d23c1418d3338251bfe469c8'
 '4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a'
 '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce'
 '4ec9599fc203d176a301536c2e091a19bc852759b255bd6818810a42c5fed14a'
 '535fa30d7e25dd8a49f1536779734ec8286108d115da5045d77f3b4185d8f790'
 '6208ef0f7750c111548cf90b6ea1d0d0a66f6bff40dbef07cb45ec436263c7d6'
 '624b60c58c9d8bfb6ff1886c2fd605d2adeb6ea4da576068201b6c6958ce93f4'
 '670671cd97404156226e507973f2ab8330d3022ca96e0c93bdbdb320c41adcaf'
 '6b86b273ff34fce19d6b804eff5a

In [11]:
df['season_holidayed_code'] = df['season_holidayed_code'].fillna(5)

In [12]:
df['state_code_residence'] = df['state_code_residence'].fillna(-1)

In [13]:
# converting to datetime object
date_col_li = ['booking_date', 'checkin_date', 'checkout_date']

for da in date_col_li:
    df[da] = pd.to_datetime(df[da],dayfirst=True,format='%d/%m/%y')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341424 entries, 0 to 341423
Data columns (total 22 columns):
booking_date                          341424 non-null datetime64[ns]
checkin_date                          341424 non-null datetime64[ns]
checkout_date                         341424 non-null datetime64[ns]
channel_code                          341424 non-null int64
main_product_code                     341424 non-null int64
numberofadults                        341424 non-null int64
numberofchildren                      341424 non-null int64
persontravellingid                    341424 non-null int64
resort_region_code                    341424 non-null int64
resort_type_code                      341424 non-null int64
room_type_booked_code                 341424 non-null int64
roomnights                            341424 non-null int64
season_holidayed_code                 341424 non-null float64
state_code_residence                  341424 non-null float64
state_code_resort 

In [15]:
# selecting object column 

object_col_li = df.select_dtypes(include='object').columns
object_col_li

Index([], dtype='object')

In [16]:
# transforming object columns

for col in object_col_li:
    df[col] = le[col].transform(df[col])

In [17]:
# No of days in advance the booking is done

df['advance_booking_day'] = (df['checkin_date']-df['booking_date']).dt.days
df.head()

,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled,advance_booking_day
0,2018-04-05,2018-04-05,2018-04-06,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,5,1,5,2,7,7.706428,0
1,2015-01-23,2015-04-11,2015-04-16,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,5,1,5,0,1,6.662563,78
2,2015-01-28,2015-02-01,2015-02-05,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,5,1,4,0,9,7.871602,4
3,2015-05-02,2015-06-11,2015-06-16,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,5,1,3,0,25,5.344943,40
4,2015-09-02,2015-12-14,2015-12-19,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,5,1,3,0,25,7.059346,103


### removing -ve advance days prediction

In [18]:

df = df[df['advance_booking_day']>-1]

### X -Y split

In [19]:
y = df.iloc[:,-2]

In [20]:
y.head()

0    7.706428
1    6.662563
2    7.871602
3    5.344943
4    7.059346
Name: amount_spent_per_room_night_scaled, dtype: float64

In [21]:
df = df.drop(df.columns[-2],axis=1)

In [22]:
df.head()

,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,advance_booking_day
0,2018-04-05,2018-04-05,2018-04-06,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,5,1,5,2,7,0
1,2015-01-23,2015-04-11,2015-04-16,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,5,1,5,0,1,78
2,2015-01-28,2015-02-01,2015-02-05,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,5,1,4,0,9,4
3,2015-05-02,2015-06-11,2015-06-16,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,5,1,3,0,25,40
4,2015-09-02,2015-12-14,2015-12-19,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,5,1,3,0,25,103


In [23]:
x = df.iloc[:,3:]

In [24]:
x.head()

,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,advance_booking_day
0,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,5,1,5,2,7,0
1,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,5,1,5,0,1,78
2,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,5,1,4,0,9,4
3,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,5,1,3,0,25,40
4,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,5,1,3,0,25,103


## Feature Engg

#### giving weightage to different codes wrt mean and median

In [74]:
channel_code_map = {1:1,2:2,3:2.1}
main_product_code_map = {4:1,7:2,3:3,2:4,1:6}
persontravellingid_map = {47:1,4752:2,46:3,4753:4,45:5,4995:7}
resort_region_code_map = {3:1,2:2,1:3}
resort_type_code_map = {3:1,7:2,0:3,2:4,4:5,1:6,5:7}
room_type_booked_code_map = {3:1,4:2,2:3,5:4,1:5,6:6}
season_holidayed_code_map = {4:1,3:2,2:3,1:4,5:5}
state_code_resort_map = {4:1,5:2,3:3,10:4,9:5,2:6,1:7,11:8,7:9,6:10,13:11}
member_age_buckets_map = {7:1,1:2,6:3,0:4,8:5,9:6,2:7,5:8,3:9,4:10}
cluster_code_map = {0:1,3:2,1:3,5:4,4:5,2:6}
reservationstatusid_code_map = {3:1,2:2,0:3,1:4}

In [75]:
change_wt_map_li = [channel_code_map,main_product_code_map,persontravellingid_map,resort_region_code_map,resort_type_code_map,room_type_booked_code_map,season_holidayed_code_map,state_code_resort_map,member_age_buckets_map,cluster_code_map,reservationstatusid_code_map]
change_wt_li = ['channel_code','main_product_code','persontravellingid','resort_region_code','resort_type_code','room_type_booked_code','season_holidayed_code','state_code_resort','member_age_buckets','cluster_code','reservationstatusid_code']
for col,wt_map in zip(change_wt_li,change_wt_map_li):
  x[col] = x[col].map(wt_map)

In [76]:
x.isnull().any()

channel_code                False
main_product_code           False
numberofadults              False
numberofchildren            False
persontravellingid          False
resort_region_code          False
resort_type_code            False
room_type_booked_code       False
roomnights                  False
season_holidayed_code       False
state_code_residence        False
state_code_resort           False
total_pax                   False
member_age_buckets          False
booking_type_code           False
cluster_code                False
reservationstatusid_code    False
resort_id                   False
advance_booking_day         False
dtype: bool

In [77]:
value_counts_li = [col for col in df.columns if df[col].nunique()<13]
value_counts_li

['channel_code',
 'main_product_code',
 'numberofchildren',
 'persontravellingid',
 'resort_region_code',
 'resort_type_code',
 'room_type_booked_code',
 'season_holidayed_code',
 'state_code_resort',
 'member_age_buckets',
 'booking_type_code',
 'cluster_code',
 'reservationstatusid_code']

In [82]:
x.head()

,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,advance_booking_day
0,2.1,6,2,0,3,1,1,1,1,3,7.0,3,3,8,1,4,2,7,0
1,1.0,6,2,0,3,1,1,2,5,3,7.0,2,2,8,1,4,3,1,78
2,1.0,6,2,0,1,3,7,2,4,3,7.0,7,2,8,1,5,3,9,4
3,1.0,6,2,2,3,2,4,1,5,3,7.0,6,2,8,1,2,3,25,40
4,1.0,6,2,0,3,2,4,2,5,3,7.0,6,2,8,1,2,3,25,103


In [84]:
df.columns

Index(['booking_date', 'checkin_date', 'checkout_date', 'channel_code',
       'main_product_code', 'numberofadults', 'numberofchildren',
       'persontravellingid', 'resort_region_code', 'resort_type_code',
       'room_type_booked_code', 'roomnights', 'season_holidayed_code',
       'state_code_residence', 'state_code_resort', 'total_pax',
       'member_age_buckets', 'booking_type_code', 'cluster_code',
       'reservationstatusid_code', 'resort_id', 'advance_booking_day'],
      dtype='object')

In [85]:
x['target'] = y

In [86]:
# printing median values 

for col in value_counts_li:
    print(str(col).center(50,"*"))
    print(x.groupby(col)['target'].median().sort_values())
    print()    

*******************channel_code*******************
channel_code
1.0    7.772013
2.1    7.946316
2.0    7.947519
Name: target, dtype: float64

****************main_product_code*****************
main_product_code
1    7.635516
2    7.766640
3    7.771113
4    7.887318
6    8.000152
Name: target, dtype: float64

*****************numberofchildren*****************
numberofchildren
12     6.020114
0      7.831014
2      7.931318
1      7.931471
8      8.042665
3      8.384021
4      8.402835
6      8.563536
5      8.596863
7      8.827044
10     8.955253
13    10.191176
Name: target, dtype: float64

****************persontravellingid****************
persontravellingid
1    7.497638
2    7.619041
3    7.799389
4    7.869805
5    7.911538
7    8.097565
Name: target, dtype: float64

****************resort_region_code****************
resort_region_code
1    7.693204
2    7.881694
3    7.923554
Name: target, dtype: float64

*****************resort_type_code*****************
resort_type_code
1    

In [89]:
x = x.drop('target',axis=1)

In [90]:
x.columns

Index(['channel_code', 'main_product_code', 'numberofadults',
       'numberofchildren', 'persontravellingid', 'resort_region_code',
       'resort_type_code', 'room_type_booked_code', 'roomnights',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'total_pax', 'member_age_buckets', 'booking_type_code', 'cluster_code',
       'reservationstatusid_code', 'resort_id', 'advance_booking_day'],
      dtype='object')

## train test split

In [25]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=42)

In [26]:
len(x_train)

324341

In [27]:
len(x_train.columns)

19

### Scaling data

In [28]:
from sklearn.preprocessing import StandardScaler

In [29]:
sc_x = StandardScaler()
sc_y = StandardScaler()

x_train_sc = sc_x.fit_transform(x_train)
y_train_sc = sc_y.fit_transform(y_train.values.reshape(-1,1))


/home/siddharth/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/siddharth/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [30]:
#y_train = y_train.values.reshape(-1,1)

In [31]:
x_test_sc = sc_x.transform(x_test)

/home/siddharth/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [32]:
y_test_sc = sc_y.transform(y_test.values.reshape(-1,1))

In [33]:
x_train_sc.shape

(324341, 19)

In [34]:
y_train_sc.shape

(324341, 1)

In [35]:
x_test_sc.shape

(17071, 19)

In [36]:
y_test_sc.shape

(17071, 1)

In [37]:
x_train_sc[:3]

array([[ 0.10698104,  0.96000662, -0.7265505 , -0.47880079, -0.19075674,
        -1.02960697, -0.76952946,  0.13648736, -1.10438231, -0.761119  ,
         0.38914865, -1.21271479, -1.02251713, -0.05338628, -0.37661748,
         0.42132801, -0.28683249,  0.65822401, -0.84711344],
       [ 1.19960122, -0.25351691, -0.15719054, -0.47880079, -0.19075674,
         1.6341436 ,  0.51935862, -1.10676384, -1.10438231, -1.88906426,
         0.38914865,  0.07554934,  1.54706696,  0.47032174, -0.37661748,
         1.01498262, -0.28683249, -0.38444842, -0.73836851],
       [ 1.19960122, -0.25351691, -0.7265505 ,  2.15742394, -0.19075674,
         0.30226832, -0.12508542,  0.13648736, -1.10438231, -0.761119  ,
        -1.07906729, -0.89064876,  0.69053893, -0.5770943 , -0.37661748,
        -0.1723266 , -0.28683249,  0.889929  ,  0.94717799]])

In [38]:
y_train[:3]

82393     6.298232
307903    8.347406
294709    9.737241
Name: amount_spent_per_room_night_scaled, dtype: float64

### Creating tensorflow code

In [ ]:
def create_placeholders(n_inputs,n_outputs):
    
    with tf.variable_scope('Placeholders'):
        x_ph = tf.placeholder(tf.float32,shape=[None,n_inputs],name='x_ph')
        
        y_ph = tf.placeholder(tf.float32,shape=[None,n_outputs],name = 'y_ph')
        
    return x_ph,y_ph
    

In [ ]:
def initialize_parameters(n_inputs,hidden_n_li,n_outputs):
    
    parameters = {}
    
    wt_in = tf.contrib.layers.variance_scaling_initializer()
    
    
    with tf.variable_scope('Weights'):
        for layer,n_neurons in enumerate(hidden_n_li,1):
            if layer == 1:
                parameters["W"+str(layer)] = tf.Variable(wt_in([n_inputs,n_neurons]),dtype=tf.float32,name="W"+str(layer))
            else:
                parameters["W"+str(layer)] = tf.Variable(wt_in([hidden_n_li[layer-2],n_neurons]),dtype=tf.float32,name="W"+str(layer))
        
        parameters["W"+str(layer+1)] = tf.Variable(wt_in([hidden_n_li[layer-1],n_outputs]),dtype=tf.float32,name="W"+str(layer+1))        
        
        
    with tf.variable_scope('Bias'):
        for layer,n_neurons in enumerate(hidden_n_li,1):
            if layer == 1:
                parameters["B"+str(layer)] = tf.Variable(np.zeros(n_neurons),dtype=tf.float32,name="B"+str(layer))
            else:
                parameters["B"+str(layer)] = tf.Variable(np.zeros(n_neurons),dtype=tf.float32,name="B"+str(layer))
        
        parameters["B"+str(layer+1)] = tf.Variable(np.zeros(n_outputs),dtype=tf.float32,name="B"+str(layer+1)) 
    
    
    return parameters

In [ ]:
def forward_propagation(x_ph,parameters):
    
    
    weights = [parameters[key] for key in sorted(parameters.keys()) if 'W' in key]
    bias = [parameters[key] for key in sorted(parameters.keys()) if 'B' in key]
    
    prev_layer = x_ph
    layers_li = []
    
    with tf.variable_scope("Layers"):        
        for layer,(wt,bi) in enumerate(zip(weights,bias),1):        
            prev_layer = tf.nn.relu(tf.add(tf.matmul(prev_layer,wt),bi),name='layer'+str(layer))        
            layers_li.append(prev_layer)
    
    return prev_layer

In [ ]:
def compute_cost(output,y_ph,parameters):    
    
    # RMSE loss
    cost = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(output,y_ph))))
    
        
    # weights tensor list
    weights = [parameters[key] for key in sorted(parameters.keys()) if "W" in key]
    
    # l2 loss
    l2_loss = 1e-6*tf.add_n([tf.nn.l2_loss(wt) for wt in weights])
    
    # final loss
    ret_cost = cost + l2_loss
    
    return ret_cost
    

In [ ]:
def get_batches(x_train,y_train,batch_size):
    
    n_batches = len(x_train)//batch_size
    
    x_train = x_train[:n_batches*batch_size]
    y_train = y_train[:n_batches*batch_size]
    
    
    for st_index in range(0,len(x_train),batch_size):
        yield x_train[st_index:st_index+batch_size],y_train[st_index:st_index+batch_size]
    
    

In [ ]:
def train_model(x_train,y_train,x_test,y_test,learning_rate,hidden_n_li,n_iteration):
    
    train_start = datetime.now()
    
    # reseting the default graph
    tf.reset_default_graph()
    
    # defining input,output parameters and batch_size
    batch_size = 2048
    
    n_batches = len(x_train)//batch_size
    
    n_inputs = len(x_train[0])
    n_outputs = len(y_train[0])
    
    # defining the placehoders
    x_ph,y_ph = create_placeholders(n_inputs,n_outputs)
    
    # defining the parameters
    parameters = initialize_parameters(n_inputs,hidden_n_li,n_outputs)
    
    # defining the forward propagation
    output = forward_propagation(x_ph,parameters)
    
    # defining cost and optimizer
    cost = compute_cost(output,y_ph,parameters)    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    [print(x.name) for x in tf.get_default_graph().get_operations() if x.type == "Placeholder"] 
    
    return
    
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    training_info = []
    nn_arch_str = ""
    for layer_neuron in hidden_n_li:
        nn_arch_str+=str(layer_neuron)+"_"
        
    saving_dir = './TF_'+nn_arch_str+str(learning_rate)+'_'+str(n_iteration)+'/pred.ckpt'
    trainingloss_sav_dir = './TF_'+nn_arch_str+str(learning_rate)+'_'+str(n_iteration)+'/trainingloss.txt'
    
    print(saving_dir)
    
    
    with tf.Session() as sess:
        sess.run(init)
        
        for epoch in range(n_iteration):
            start = datetime.now()
            
            av_cost = []
            
            for step,(x_b,y_b) in enumerate(get_batches(x_train,y_train,batch_size),1):
                print('.',end= ' ')
                
                feed = {x_ph:x_b,
                        y_ph:y_b}
                
                cost_,_ = sess.run([cost,optimizer],feed_dict=feed)
                av_cost.append(cost_)
                
                if step== n_batches:
                    stop = datetime.now()
                    
                    feed_test = {x_ph:x_test,
                                 y_ph:y_test}
                    
                    test_cost = sess.run(cost,feed_dict = feed_test)
                    
                    print()
                    information = "Epoch : {:04d}/{}... Batch : {}/{}... test_cost : {}... train_Cost : {}... Epoch time : {}".format(epoch+1,n_iteration,step,n_batches,test_cost,np.mean(av_cost),str(stop-start))
                    training_info.append(information)
                    print(information)
                
            if epoch ==0:
                prev_loss = np.mean(av_cost)
                continue
                
            elif np.mean(av_cost)<prev_loss:             
                saver.save(sess,saving_dir,global_step = epoch+1)
                prev_loss = np.mean(av_cost)
        
        train_stop = datetime.now()
        print("Total training time : ",str(train_stop-train_start))
                
        
        with open(trainingloss_sav_dir,'w') as f:
            for info in training_info:
                f.write('%s\n' %info)
            f.write('Total training time : %s' %str(train_stop-train_start))
            
    

In [ ]:
lr_li = np.linspace(1e-5,1e-2,num = 20)
lr_li

In [ ]:
# driver function

lr_li = [0.01,0.001,0.0001,0.00001,0.000001]

#for lr in lr_li:
lr=1    
train_model(x_train_sc,y_train_sc,x_test_sc,y_test_sc,learning_rate = lr,hidden_n_li=[50,100,75],n_iteration=50)

In [ ]:
# with open('sc_x_train','wb') as f:
#     pkl.dump(sc_x,f)

In [ ]:
# with open('sc_y_train','wb') as f:
#     pkl.dump(sc_y,f)

## Predicting test.csv 

In [39]:
test_df = pd.read_csv('./dataset/test.csv')

In [40]:
test_df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d4787cb4596e3086b4f,05/04/18,05/04/18,06/04/18,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,A,C,9f14025af0065b30e47e23ebb3b491d39ae8ed17d33739e5ff3827ffb3634953
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb1946e644b405d6233c,24/05/15,23/06/15,27/06/15,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,F,A,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e38f40dcb4ce0c4f552,21/07/15,07/08/15,10/08/15,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd26796a33cc98b01c6095c4,31/07/15,22/10/15,25/10/15,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,F,A,49d180ecf56132819571bf39d9b7b342522a2ac6d23c1418d3338251bfe469c8
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1b2d117cd6602737ec1,01/07/16,01/10/16,05/10/16,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,F,A,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027aa13515e4f3884bb6b


In [41]:
test_df.isnull().sum()

reservation_id                 0
booking_date                   0
checkin_date                   0
checkout_date                  0
channel_code                   0
main_product_code              0
numberofadults                 0
numberofchildren               0
persontravellingid             0
resort_region_code             0
resort_type_code               0
room_type_booked_code          0
roomnights                     0
season_holidayed_code         35
state_code_residence        2260
state_code_resort              0
total_pax                      0
member_age_buckets             0
booking_type_code              0
memberid                       0
cluster_code                   0
reservationstatusid_code       0
resort_id                      0
dtype: int64

In [42]:
test_df['season_holidayed_code'] = test_df['season_holidayed_code'].fillna(5)

In [43]:
test_df['state_code_residence'] = test_df['state_code_residence'].fillna(-1)

In [44]:
# converting to datetime object
date_col_li = ['booking_date', 'checkin_date', 'checkout_date']

for da in date_col_li:
    test_df[da] = pd.to_datetime(test_df[da],dayfirst=True,format='%d/%m/%y')

In [45]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146765 entries, 0 to 146764
Data columns (total 23 columns):
reservation_id              146765 non-null object
booking_date                146765 non-null datetime64[ns]
checkin_date                146765 non-null datetime64[ns]
checkout_date               146765 non-null datetime64[ns]
channel_code                146765 non-null int64
main_product_code           146765 non-null int64
numberofadults              146765 non-null int64
numberofchildren            146765 non-null int64
persontravellingid          146765 non-null int64
resort_region_code          146765 non-null int64
resort_type_code            146765 non-null int64
room_type_booked_code       146765 non-null int64
roomnights                  146765 non-null int64
season_holidayed_code       146765 non-null float64
state_code_residence        146765 non-null float64
state_code_resort           146765 non-null int64
total_pax                   146765 non-null int64
member_

In [46]:
# selecting object column 

object_col_li = test_df.select_dtypes(include='object').columns
object_col_li = object_col_li[1:]
object_col_li

Index(['member_age_buckets', 'memberid', 'cluster_code',
       'reservationstatusid_code', 'resort_id'],
      dtype='object')

In [47]:
# removing memberid

object_col_li = [ col for col in object_col_li if 'memberid' not in col]
object_col_li

['member_age_buckets', 'cluster_code', 'reservationstatusid_code', 'resort_id']

In [48]:
test_df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d4787cb4596e3086b4f,2018-04-05,2018-04-05,2018-04-06,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,A,C,9f14025af0065b30e47e23ebb3b491d39ae8ed17d33739e5ff3827ffb3634953
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb1946e644b405d6233c,2015-05-24,2015-06-23,2015-06-27,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,F,A,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e38f40dcb4ce0c4f552,2015-07-21,2015-08-07,2015-08-10,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd26796a33cc98b01c6095c4,2015-07-31,2015-10-22,2015-10-25,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,F,A,49d180ecf56132819571bf39d9b7b342522a2ac6d23c1418d3338251bfe469c8
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1b2d117cd6602737ec1,2016-07-01,2016-10-01,2016-10-05,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,H,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,F,A,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027aa13515e4f3884bb6b


In [49]:
# transforming object columns

for col in object_col_li:
    test_df[col] = le[col].transform(test_df[col])
    print('done for ',col)

done for  member_age_buckets
done for  cluster_code
done for  reservationstatusid_code
done for  resort_id


In [50]:
# No of days in advance the booking is done

test_df['advance_booking_day'] = (test_df['checkin_date']-test_df['booking_date']).dt.days
test_df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,advance_booking_day
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d4787cb4596e3086b4f,2018-04-05,2018-04-05,2018-04-06,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,7,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,0,2,20,0
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb1946e644b405d6233c,2015-05-24,2015-06-23,2015-06-27,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,7,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,5,0,28,30
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e38f40dcb4ce0c4f552,2015-07-21,2015-08-07,2015-08-10,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,7,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,3,0,25,17
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd26796a33cc98b01c6095c4,2015-07-31,2015-10-22,2015-10-25,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,7,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,5,0,5,83
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1b2d117cd6602737ec1,2016-07-01,2016-10-01,2016-10-05,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,7,1,2114944930dcc42ce5b9b50ae965cf8a9c04e46be63d8429af7996855fec3b12,5,0,27,92


In [51]:
# # no of days stay

# test_df['days_stay'] = (test_df['checkout_date']-test_df['checkin_date']).dt.days
# test_df.head()

In [52]:
# dropping memberid

test_df = test_df.drop('memberid',axis=1)

In [53]:
test_df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,advance_booking_day
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d4787cb4596e3086b4f,2018-04-05,2018-04-05,2018-04-06,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,7,1,0,2,20,0
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb1946e644b405d6233c,2015-05-24,2015-06-23,2015-06-27,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,7,1,5,0,28,30
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e38f40dcb4ce0c4f552,2015-07-21,2015-08-07,2015-08-10,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,7,1,3,0,25,17
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd26796a33cc98b01c6095c4,2015-07-31,2015-10-22,2015-10-25,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,7,1,5,0,5,83
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1b2d117cd6602737ec1,2016-07-01,2016-10-01,2016-10-05,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,7,1,5,0,27,92


In [54]:
wrong_yr_index = test_df[test_df['advance_booking_day']<0].index
wrong_yr_index

Int64Index([41, 57], dtype='int64')

In [55]:
test_df.loc[wrong_yr_index]['checkin_date']


41   2012-04-20
57   2012-04-28
Name: checkin_date, dtype: datetime64[ns]

In [56]:
test_df.loc[wrong_yr_index]['checkin_date']+pd.DateOffset(years=6)

41   2018-04-20
57   2018-04-28
Name: checkin_date, dtype: datetime64[ns]

In [57]:
test_df.at[wrong_yr_index,['checkin_date']] = test_df.iloc[wrong_yr_index]['checkin_date']+pd.DateOffset(years=6)
test_df.loc[wrong_yr_index]

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,advance_booking_day
41,6ec2dd92d278aa97f8ab095ac0c0d18a7e9f18753c51c798a25c2d3a7f816212,2018-04-06,2018-04-20,2012-04-24,3,3,2,0,4752,2,2,2,2,3.0,8.0,2,4,7,1,3,2,19,-2177
57,54bf09a310761d2f874666557e3706012d85a47b14d0b8e5bd393344335b3b6f,2018-04-06,2018-04-28,2012-05-01,3,1,3,0,45,1,1,3,7,2.0,8.0,4,3,4,2,5,2,28,-2169


In [58]:
test_df.at[wrong_yr_index,['checkout_date']] = test_df.iloc[wrong_yr_index]['checkout_date']+pd.DateOffset(years=6)
test_df.loc[wrong_yr_index]

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,advance_booking_day
41,6ec2dd92d278aa97f8ab095ac0c0d18a7e9f18753c51c798a25c2d3a7f816212,2018-04-06,2018-04-20,2018-04-24,3,3,2,0,4752,2,2,2,2,3.0,8.0,2,4,7,1,3,2,19,-2177
57,54bf09a310761d2f874666557e3706012d85a47b14d0b8e5bd393344335b3b6f,2018-04-06,2018-04-28,2018-05-01,3,1,3,0,45,1,1,3,7,2.0,8.0,4,3,4,2,5,2,28,-2169


In [59]:
# No of days in advance the booking is done

test_df['advance_booking_day'] = (test_df['checkin_date']-test_df['booking_date']).dt.days
test_df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,advance_booking_day
0,7dae1ce6bc8f69481328f2be5c4943077dad5598b5f66d4787cb4596e3086b4f,2018-04-05,2018-04-05,2018-04-06,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,7,1,0,2,20,0
1,fe0d4e444e1818436c88f72f1cf800536c2f785e59baeb1946e644b405d6233c,2015-05-24,2015-06-23,2015-06-27,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,7,1,5,0,28,30
2,540bd4285ad8168e8388f84ee74a82cd4f97dc0a404d3e38f40dcb4ce0c4f552,2015-07-21,2015-08-07,2015-08-10,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,7,1,3,0,25,17
3,09593c907ae262e46f655b4db9e14f54a19eadcfdd26796a33cc98b01c6095c4,2015-07-31,2015-10-22,2015-10-25,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,7,1,5,0,5,83
4,f4c50caac68051faf37551d70bb17eebef2a20e2244cb1b2d117cd6602737ec1,2016-07-01,2016-10-01,2016-10-05,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,7,1,5,0,27,92


In [60]:
test_df[test_df['advance_booking_day']<0]

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,advance_booking_day


In [61]:
xx = test_df.iloc[:,4:]

In [62]:
xx.head()

,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,advance_booking_day
0,3,2,2,0,45,3,3,4,1,2.0,2.0,3,3,7,1,0,2,20,0
1,1,2,4,0,45,1,1,4,8,2.0,2.0,4,2,7,1,5,0,28,30
2,3,2,3,0,45,2,2,2,3,4.0,2.0,2,3,7,1,3,0,25,17
3,1,2,3,0,45,1,1,2,3,2.0,2.0,6,3,7,1,5,0,5,83
4,1,2,4,0,45,1,2,3,8,2.0,2.0,11,2,7,1,5,0,27,92


In [63]:
len(xx)

146765

In [64]:
len(xx.columns)

19

In [65]:
xx_sc = sc_x.transform(xx)

/home/siddharth/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [66]:
xx_sc.shape

(146765, 19)

In [71]:
# load model

def load_model(x_test):

    pred_st_time = datetime.now()
    
    
    tf.reset_default_graph()

    with tf.Session() as sess:    
        
        # loading model
        saver = tf.train.import_meta_graph("TF_50_100_200_70_10_1.2742749857031348e-05_200/pred.ckpt-200.meta")
        saver.restore(sess,"TF_50_100_200_70_10_1.2742749857031348e-05_200/pred.ckpt-200")

        graph = tf.get_default_graph()
        
        # getting all tensors
        x_ph = graph.get_tensor_by_name('Placeholders/x_ph:0')
        outputs = graph.get_tensor_by_name('Layers/layer6:0')
        
        [print(x.name) for x in tf.get_default_graph().get_operations() if x.type == "Placeholder"] 

        print(x_ph)
        # getting logits
        feed_test = {x_ph:x_test}
        logits = sess.run(outputs,feed_dict=feed_test)
        
        #print(logits.shape)
        
        pred_stop_time = datetime.now()
        print("Prediction time : ",str(pred_stop_time-pred_st_time))
                
        return logits

In [72]:
out_test = load_model(xx_sc)

INFO:tensorflow:Restoring parameters from TF_50_100_200_70_10_1.2742749857031348e-05_200/pred.ckpt-200
Placeholders/x_ph
Placeholders/y_ph
Tensor("Placeholders/x_ph:0", shape=(?, 19), dtype=float32)
Prediction time :  0:00:00.528931


In [ ]:
out_test.shape

In [ ]:
final_pred = out_test

In [ ]:
df_pred = pd.DataFrame()

In [ ]:
test_df.head()

In [ ]:
df_pred['reservation_id'] = test_df['reservation_id']

In [ ]:
df_pred.info()

In [ ]:
df_pred.head()

In [ ]:
df_pred['amount_spent_per_room_night_scaled'] = final_pred

In [ ]:
df_pred[:1000]

In [ ]:
#df_pred.to_csv('pred2.csv',index=False)